In [1]:
# Set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# Import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from pyimagesearch.nn.conv import ResNet
from pyimagesearch.callbacks import TrainingMonitor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.datasets import cifar10
import numpy as np
import argparse
import sys
import os
from tensorflow.keras.utils import plot_model


2024-12-05 23:20:50.297231: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 23:20:50.305083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733422850.314360    5141 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733422850.317065    5141 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 23:20:50.327053: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# set a high recursion limit so Theano doesn’t complain
sys.setrecursionlimit(5000)

# define the total number of epochs to train for along with the
# initial learning rate
NUM_EPOCHS = 100
INIT_LR = 1e-1

def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0

    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power

    # return the new learning rate
    return alpha

In [3]:
# Define the arguments dictionary
args = {
    "output": "output",
    "model": "output/resnet_cifar10.hdf5"
}

In [4]:

# load the training and testing data, converting the images from
# integers to floats
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1, 
                         horizontal_flip=True,
                         fill_mode="nearest")

# construct the set of callbacks
figPath = os.path.sep.join([args["output"], "{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([args["output"], "{}.json".format(os.getpid())])
callbacks = [TrainingMonitor(figPath, jsonPath=jsonPath),
             LearningRateScheduler(poly_decay)]

# initialize the optimizer and model (ResNet-56)

[INFO] loading CIFAR-10 data...


In [5]:

# initialize the optimizer and model (ResNet-56)
print("[INFO] compiling model...")
opt = SGD(learning_rate=INIT_LR, momentum=0.9)
model = ResNet.build(32, 32, 3, 10, (9, 9, 9),
                     (64, 64, 128, 256), reg=0.0005)

# model = ResNet.build(32, 32, 3, 10, (1,1),
#                      (64, 64, 128), reg=0.0005)

plot_model(model, to_file="resnet.png", show_shapes=True)

model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
              metrics=["accuracy"])




[INFO] compiling model...


I0000 00:00:1733422853.127735    5141 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1164 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
# Train the network
print("[INFO] training network...")
history = model.fit(
    aug.flow(trainX, trainY, batch_size=32),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // 128,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)




[INFO] training network...
Epoch 1/10


/home/umair-ali/Desktop/phd deep learning after mid/codes to make operational and understand/tf_phd/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1733422865.154479    5269 service.cc:148] XLA service 0x7395cc0b8ad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733422865.154500    5269 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-12-05 23:21:05.562129: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733422866.678684    5269 cuda_

  2/390 ━━━━━━━━━━━━━━━━━━━━ 29s 76ms/step - accuracy: 0.1719 - loss: 2.7444   

I0000 00:00:1733422892.935133    5269 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


390/390 ━━━━━━━━━━━━━━━━━━━━ 81s 110ms/step - accuracy: 0.2133 - loss: 3.2781 - val_accuracy: 0.3146 - val_loss: 2.9195 - learning_rate: 0.1000
Epoch 2/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 35s 91ms/step - accuracy: 0.3867 - loss: 2.5144 - val_accuracy: 0.4736 - val_loss: 2.1592 - learning_rate: 0.0990
Epoch 3/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.4723 - loss: 2.1010

In [ ]:
# save the network to disk
print("[INFO] serializing network...")
model.save(args["model"])
